# Notebook for Gene Set Score Analysis - Mouse Heart Cardiomyocytes

**Created by** : **Srivalli Kolla**

**Devloped on** : October 7 ,2024 

**Last Modified** : November 21 ,2024 

**Institute of Systems Immunology, University of Wurzburg**

Env : scanpy

# Importing packages

In [6]:
import anndata
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from SCCAF import SCCAF_assessment, plot_roc

## Set up working environment

In [7]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 300, color_map = 'RdPu', dpi_save = 300, vector_friendly = True, format = 'png')

timestamp = datetime.datetime.now().strftime("%d_%m_%y,%H:%M")
timestamp

# Data Loading and Preprocessing

In [8]:
cmc = sc.read_h5ad('../data/dmd_annotated_cmc_concatenated_19_11_24,16:32.h5ad')
cmc

In [9]:
cmc.obs['cell_type'].value_counts()

In [10]:
cmc.obs['genotype'].value_counts()

In [11]:
cmc.obs['cell_state'].value_counts()

In [12]:
cmc = cmc[cmc.obs['cell_state'] != 'vCM5']
cmc

In [13]:
pd.crosstab(cmc.obs['cell_state'],cmc.obs['sample'])

In [14]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

X_is_raw(cmc)

In [15]:
sc.pp.normalize_total(cmc, target_sum=1e6)
sc.pp.log1p(cmc)

X_is_raw(cmc)

In [16]:
sc.pp.neighbors(cmc,  n_neighbors = 150, metric = 'minkowski')
sc.tl.umap(cmc, min_dist = 0.3, spread = 4, random_state = 1712)

In [17]:
sc.pl.umap(cmc, frameon = False, color = ['sample', 'genotype', 'cell_state'], size = 2.5, legend_fontsize = 5, ncols = 3)

# Gene set scoring

## Interferon gaama signalling

In [18]:
interferon_genes = ['Stat1',
'Ifng',
'Jak1',
'Tyk2',
'Tp53',
'Ifngr1',
'Irf1',
'Jak2',
'Ifngr2',
'Hpx']

In [19]:
sc.tl.score_genes(cmc,
                  gene_list = interferon_genes,
                  score_name = 'interferon_score',
                  random_state = 1712)

In [20]:
sc.pl.umap(cmc, 
           frameon = False, 
           color = ['genotype', 'cell_state','interferon_score'], 
           size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r')

In [21]:
present_interferon = cmc.var.index[cmc.var.index.isin(interferon_genes)]
present_interferon = present_interferon.astype(str)
present_interferon

In [22]:
sc.pl.dotplot(cmc, present_interferon,groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma', title= 'interferon_cmcs_cell_state', save= 'interferon_cmcs_cell_state.png')

In [23]:
sc.pl.dotplot(cmc, present_interferon,groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma',title= 'interferon_cmcs_genotype',save= 'interferon_cmcs_genotype.png')

## Apoptosis

In [24]:
apoptotic_genes = ['Daxx',
'Col6a3',
'Casp8',
'Gapdh',
'Pfkfb3',
'Tgfb2',
'Pdcl3',
'Snw1',
'Sh3glb1',
'Nradd',
'Clca3a2',
'Diablo',
'Bok',
'Babam2',
'Bcl2l13',
'Egln3',
'Dap3',
'Madd',
'Trp63',
'1600014C10Rik',
'Tfpt',
'Cib1',
'Bcl2l12',
'Trp53',
'Bax',
'Bbc3',
'Ppp1r15a',
'Tmem238l',
'Gm45808',
'Unc5d',
'Bag6',
'Gapdhrt2',
'Elmo3',
'Ppia',
'Birc6',
'Dffb',
'Cflar',
'Stk3',
'Ripk3',
'Bcl2l2',
'Ddit3',
'Bik',
'Tmbim6',
'Tcirg1',
'Sgms1',
'Pmaip1',
'Bad',
'Fas',
'Scml2',
'Zdhhc3',
'Ube4b',
'Casp7',
'Pten',
'Xaf1',
'Rad21',
'Fasl',
'Tnfrsf9',
'Nptx1',
'Bcl2l1',
'Gdf6',
'Rnf186',
'Map3k7',
'Tpx2',
'Casp9',
'Agrn',
'Bmf',
'Birc7',
'Ebf4',
'A5JTV6',
'A6YM30',
'Aifm1',
'Scn2a',
'Tnfrsf25',
'Casp8ap2',
'Trp73',
'Ift57',
'Pde3a',
'Tradd',
'Rrn3',
'Ep300',
'Taok1',
'Rbm25',
'Syce3',
'Ddit4',
'Bcl10',
'Tmf1',
'Cit',
'Gramd4',
'Rps3',
'Slc5a11',
'Tnfaip8',
'Rps27l',
'Bcl2l14',
'Dedd',
'Cidec',
'Pea15a',
'Dnase2a',
'Gm20521',
'Opa1',
'Pdcd10',
'Ciapin1',
'Topaz1',
'Dapk2',
'Rrp8',
'Unc5c',
'Dlc1',
'Fhit',
'Cecr2',
'Ryr2',
'Tgfbr1',
'Coa8',
'Trp53bp2',
'Brat1',
'Bnipl',
'Fgfr3',
'Vdac1',
'Gm29094',
'Syngap1',
'Cdip1',
'Becn1',
'Bak1',
'Prune2',
'Ercc6',
'Il12a',
'Traf7',
'Ppp2r1b',
'Slit2',
'Qrich1',
'Zfp212',
'Apaf1',
'CYCt',
'Senp1',
'Hk2',
'Esr2',
'Angpt1',
'Eya2',
'Stxbp1',
'Endog',
'Mknk1',
'Gzmm',
'Bcl7c',
'Tnfrsf11b',
'Akap1',
'Casp12',
'Casp6',
'Ercc2',
'Birc3',
'Bub1',
'Gcm2',
'Ntn1',
'Gclm',
'Map3k5',
'Atn1',
'Cx3cl1',
'Chek1',
'Xbp1',
'Sema6a',
'Fkbp8',
'C1d',
'Notch2',
'Rabep1',
'Fgf10',
'Bmpr2',
'Axin1',
'C1qbp',
'Tnfrsf18',
'Api5',
'Emp3',
'Dapk3',
'Dffa',
'Csnk2a2',
'Zmat3',
'Tnfsf12',
'Bcl2l11',
'Siva1',
'Bnip2',
'Lgals7',
'Slk',
'Bnip3',
'Snca',
'Dnase1l3',
'Bcl2a1b',
'A1c',
'Bcl2a1d',
'Cd70',
'Birc5',
'Cidea',
'Cideb',
'Gpx4',
'Ppargc1a',
'Sfn',
'Cfdp1',
'Tgfbr3',
'Foxl2',
'Cntfr',
'Dnmt3a',
'Ikbkg',
'Parp2',
'Men1',
'Pglyrp1',
'Pak1',
'Emp2',
'Jtb',
'Cradd',
'Tp63',
'Hipk1',
'Coro1a',
'Abl1',
'Myc',
'Ngf',
'Ins2',
'Ifng',
'Il1a',
'Il3',
'Il2ra',
'Tp53',
'Afp',
'Ada',
'Gzmb',
'Tgfb1',
'Il2',
'Igf1',
'Itgam',
'Jun',
'Krt18',
'Ptprc',
'Enpp1',
'Pim1',
'Tnf',
'Epo',
'Anxa2',
'Il4',
'Hsp90aa1',
'Sod1',
'Il6',
'Nefl',
'Sod2',
'Ptcra',
'Ifi204',
'Rbm10',
'Il7',
'Bcl2',
'Il1b',
'Parp1',
'Gpx1',
'Fgf4',
'Cdk1',
'Dmd',
'Lcn2',
'Krt8',
'Camk2a',
'Agt',
'Gas2',
'Itpr1',
'App',
'Rapsn',
'Nr4a1',
'Pdcd6',
'Gsn',
'Msx1',
'Cd5',
'Rb1',
'Srgn',
'Mecom',
'Anxa6',
'Hmox1',
'Irf1',
'Spn',
'Usp53',
'Lgals1',
'Atf2',
'Hmga1',
'Tgfb3',
'Pou4f1',
'Spi1',
'Gba1',
'Gata1',
'Ccn1',
'Rarg',
'Lsp1',
'Id1',
'Ntf3',
'Prkca',
'Fcer1g',
'Kitlg',
'Bdnf',
'Bmp4',
'Fgfr2',
'Gadd45b',
'Hnf1a',
'Myd88',
'Wt1',
'Rarb',
'Cd3e',
'Wnt4',
'Wnt5a',
'Bmp7',
'Pde6b',
'Pnp',
'Pcgf2',
'Mdm2',
'Cryab',
'Zfp36l1',
'Wnt7a',
'Aldh1a1',
'Cryaa',
'Cd24',
'Tnfrsf1a',
'Tnfrsf1b',
'Nfkb1',
'Bmi1',
'Ctnna1',
'Pik3r1',
'Marcks',
'S100a8',
'Pdia3',
'Lef1',
'Hnf1b',
'Braf',
'Cebpb',
'Wnt7b',
'Col2a1',
'Max',
'Camk2b']

In [25]:
sc.tl.score_genes(cmc,
                  gene_list = apoptotic_genes,
                  score_name = 'apoptotic_score',
                  random_state = 1712)

In [26]:
sc.pl.umap(cmc, 
           frameon = False, 
           color = ['genotype', 'cell_state', 'apoptotic_score'], 
           size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r')

In [27]:
present_apoptotic = cmc.var.index[cmc.var.index.isin(apoptotic_genes)]
present_apoptotic = present_apoptotic.astype(str)
present_apoptotic

In [28]:
expression_matrix = (cmc[:, present_apoptotic].X > 0).toarray()
expression_df = pd.DataFrame(expression_matrix, index=cmc.obs.index, columns=present_apoptotic)
expression_df['cell_state'] = cmc.obs['cell_state'].values

fraction_expressed = expression_df.groupby('cell_state').mean()


genes_above_threshold = (fraction_expressed.max(axis=0) > 0.4)
filtered_apoptotic = present_apoptotic[genes_above_threshold.values]


In [29]:
sc.pl.dotplot(cmc, filtered_apoptotic ,groupby= 'cell_state',
              dendrogram = True, 
              cmap = 'magma', title = 'apoptotic_cmcs_cell_state',save= 'apoptotic_cmcs_cell_state.png')

In [30]:
sc.pl.dotplot(cmc, filtered_apoptotic ,groupby= 'genotype', 
              dendrogram = True,
              cmap = 'magma',  title= 'apoptotic_cmcs_genotype',save= 'apoptotic_cmcs_genotype.png')

## Ferroptosis

In [31]:
ferroptosis_genes = ["Ninj1",
    "Tp53",
    "Hmox1"
    "Gpx4",
    "Slc7a11",
    "Acsl4",
    "Tfrc",
    "Fth1",
    "Ftl",
    "Ncoa4",
    "Lpcat3",
    "Alox15",
    "Ptgs2",
    "Nfe2l2",
    "Cisd1",
    "Sat1",
    "Rsl",
    "Gls2",
    "Dpp4"]

In [32]:
sc.tl.score_genes(cmc,
                  gene_list = ferroptosis_genes,
                  score_name = 'ferroptosis_score',
                  random_state = 1712)

In [33]:
sc.pl.umap(cmc, 
           frameon = False, 
           color = ['genotype', 'cell_state','ferroptosis_score'], 
           size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r')

In [34]:
present_ferroptosis = cmc.var.index[cmc.var.index.isin(ferroptosis_genes)]
present_ferroptosis = present_ferroptosis.astype(str)
present_ferroptosis

In [35]:
sc.pl.dotplot(cmc,present_ferroptosis,groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma',title = 'ferroptosis_cmcs_cell_state', save= 'ferroptosis_cmcs_cell_state.png')

In [36]:
sc.pl.dotplot(cmc, present_ferroptosis,groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title= 'ferroptosis_cmcs_genotype',save= 'ferroptosis_cmcs_genotype.png')

## Oxidative Phosphorylation

In [37]:
phosphorylation_genes = ['ND5',
'CYTB',
'ND4',
'ND2',
'COX2',
'ND4L',
'COX1',
'COX3',
'COXI',
'Ndufa7',
'Ndufs1',
'COI',
'Cox5b',
'COXII',
'COXIII',
'Ndufa12',
'Ndufc2',
'Cox7a1',
'Ndufa8',
'Ndufs6',
'Cox8a',
'Sdhaf2',
'Ndufb6',
'Cox4i1',
'Cox8c',
'Proca1',
'Cox7c',
'Co3',
'C0JZQ9',
'C0JZR0',
'Cox7a2l',
'Atp5e',
'Tafazzin',
'Uqcrq',
'Dld',
'Ndufb11',
'Ndufa1',
'Fxn',
'Snca',
'Mt-Cyb',
'Mtco1',
'mt-Co3',
'Mtatp6',
'Mtnd1',
'mt-Nd2',
'mt-Nd3',
'Mtnd4l',
'Mtnd4',
'Mtnd5',
'Mtnd6',
'Ndufb1',
'Cox5a',
'Cox6a2',
'Cox6a1',
'Msh2',
'Cox7a2',
'Cox8b',
'Atp5mf',
'Atp5f1e',
'Cox6b1',
'Cox7b',
'Atp5f1b',
'Bid',
'Atp5pf',
'Coq7',
'Uqcrh',
'Atp5f1a',
'Atp5me',
'mt-Co1',
'Q3TRR5',
'Atp5f1c',
'Uqcr11',
'mt-Nd4',
'Q3UQ68',
'Q3UR21',
'Q3UVI0',
'Uqcc3',
'Afg1l',
'Atp5f1d',
'mt-Nd1',
'Q5BP84',
'Q5BP85',
'Slc25a51',
'Atp5pb',
'Ndufs8',
'Uqcr10',
'Tefm',
'mt-Nd6',
'Q6TM11',
'Q6TM12',
'Q71EJ0',
'Dnajc15',
'Mtch2',
'ATP6',
'Nipsnap2',
'Chchd10',
'Cox6b2',
'Coa6',
'Ndufv3',
'Coq9',
'Sdha',
'Cox4i2',
'Ndufs2',
'Ndufv1',
'Stoml2',
'Ndufa10',
'Ndufs5',
'Ndufa5',
'Cox6c',
'Ndufb2',
'Ndufa2',
'Ndufa3',
'Sdhb',
'Uqcrb',
'Ndufb4',
'Ndufb5',
'Ndufb9',
'Ndufc1',
'Ndufa6',
'Ndufb3',
'Ndufab1',
'Ndufb7',
'Sdhd',
'Ndufs4',
'Uqcrc1',
'Sdhc',
'Cyc1',
'Cox7b2',
'Ndufb8',
'Ndufv2',
'Cox5b-ps',
'Ndufa11',
'Q9D9H8',
'Atp5po',
'Uqcrc2',
'Ndufa9',
'Ndufs7',
'Ndufb10',
'Ndufs3',
'Atp5pd',
'Ndufa13',
'mt-Nd4l',
'mt-Nd5']

In [38]:
sc.tl.score_genes(cmc,
                  gene_list = phosphorylation_genes,
                  score_name = 'phosphorylation_score',
                  random_state = 1712)

In [39]:
sc.pl.umap(cmc, 
           frameon = False, 
           color = ['sample', 'genotype','phosphorylation_score'], 
           size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r')

In [40]:
present_phosphorylation = cmc.var.index[cmc.var.index.isin(phosphorylation_genes)]
present_phosphorylation = present_phosphorylation.astype(str)
present_phosphorylation

In [41]:
sc.pl.dotplot(cmc, present_phosphorylation,groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma',title = 'phosphorylation_cmcs_cell_state', save= 'phosphorylation_cmcs_cell_state.png')

In [42]:
sc.pl.dotplot(cmc, present_phosphorylation,groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='phosphorylation_cmcs_genotype',save= 'phosphorylation_cmcs_genotype.png')

## Glycolysis

In [43]:
glycolysis_genes = ['Pgam2',
'Aldoa',
'Gpi',
'Pgk1',
'Gapdh',
'Eno1',
'Hk1',
'Tpi1',
'Eno3',
'Foxk1',
'Pfkm',
'Pkm',
'Pgam1',
'Foxk2']

In [44]:
sc.tl.score_genes(cmc,
                  gene_list = glycolysis_genes,
                  score_name = 'glycolysis_score',
                  random_state = 1712)

In [45]:
sc.pl.umap(cmc, 
           frameon = False, 
           color = [ 'genotype', 'cell_state','glycolysis_score'], 
           size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r')

In [46]:
present_glycolysis_genes = cmc.var.index[cmc.var.index.isin(glycolysis_genes)]
present_glycolysis_genes = present_glycolysis_genes.astype(str)
present_glycolysis_genes

In [47]:
sc.pl.dotplot(cmc, present_glycolysis_genes,groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma',title = 'glycolysis_cmcs_cell_state', save= 'glycolysis_cmcs_cell_state.png')

In [48]:
sc.pl.dotplot(cmc,present_glycolysis_genes,groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='glycolysis_cmcs_genotype',save= 'glycolysis_cmcs_genotype.png')

## Pyroptosis

In [49]:
pyroptosis_genes = ['Gsdmd',
'Ninj1',
'Gzma',
'Casp4',
'Gsdme']

In [50]:
sc.tl.score_genes(cmc,
                  gene_list = pyroptosis_genes,
                  score_name = 'pyroptosis_score',
                  random_state = 1712)

In [51]:
sc.pl.umap(cmc, 
           frameon = False, 
           color = [ 'genotype', 'cell_state','pyroptosis_score'], 
           size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r')

In [52]:
present_pyroptosis_genes = cmc.var.index[cmc.var.index.isin(pyroptosis_genes)]
present_pyroptosis_genes = present_pyroptosis_genes.astype(str)
present_glycolysis_genes

In [53]:
sc.pl.dotplot(cmc, present_glycolysis_genes,groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma',title = 'pyroptosis_cmcs_cell_state', save= 'pyroptosis_cmcs_cell_state.png')

In [54]:
sc.pl.dotplot(cmc,present_glycolysis_genes,groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='pyroptosis_cmcs_genotype',save= 'pyroptosis_cmcs_genotype.png')

## Fatty acid metabolism

In [55]:
fatty_acid_genes = ['Hsd17b8',
'Gcdh',
'Acad9',
'Elovl6',
'Lias',
'Acox3',
'Alox5',
'Acadl',
'Acad11',
'Acad8',
'Adtrp',
'Elovl2',
'Fasn',
'Fads3',
'Cyp2e1',
'Cpt1c',
'Acsf3',
'Ilvbl',
'Gm49366',
'Acot3',
'Elovl7',
'Mcat',
'Prkag1',
'Decr2',
'Fads1',
'Cpt1a',
'Scd2',
'Elovl3',
'Tnxb',
'Tecr',
'Acox1',
'Cpt2',
'Cyp4a12b',
'Hsd17b10',
'Acnat1',
'Hacd4',
'Hacd1',
'Acss2',
'Acot11',
'Alox12',
'Alox15',
'Acot12',
'Cryl1',
'Acsl6',
'Acadvl',
'Alox8',
'Aldh3a2',
'Cyp4v3',
'Acot6',
'Crat',
'Prkag2',
'Hacl1',
'Pccb',
'Acsl1',
'Elovl5',
'Ptges3',
'Acad12',
'Ptgr2',
'Prkag3',
'Acsl3',
'Gm20431',
'Acot7',
'Acacb',
'Cyp2c23',
'Acadsb',
'Acox2',
'Slc27a6',
'Pnpla3',
'Fads2',
'Tnfrsf1a',
'Tbxas1',
'Ndufab1',
'Ech1',
'Bdh2',
'Slc27a3',
'Naaa',
'Aloxe3',
'Faah',
'Degs1',
'Pex5',
'Ptgds',
'Amacr',
'Ptgis',
'Aoah',
'Phyh',
'Slc27a2',
'Mgll',
'Cyp4a14',
'Per2',
'Snca',
'Acot1',
'Gpx4',
'Hsd17b12',
'Slc25a17',
'Alox12b',
'Comt',
'Cyp4a10',
'Cyp1a1',
'Cyp1a2',
'C3',
'Cd74',
'Enpp1',
'Tyrp1',
'Gstm1',
'Lpl',
'Gpx1',
'Nudt19',
'Scd1',
'Gsta1',
'Gstm2',
'Gstp1',
'Akr1b7',
'Hnf1a',
'Edn1',
'Edn2',
'Ptgs1',
'Ppara',
'Th',
'Plaa',
'Lipc',
'Adh5',
'Alox5ap',
'Prkar2b',
'Pla2g2a',
'Scp2',
'Mif',
'Ppard',
'Lep',
'Eci1',
'Akr1b1',
'Acadm',
'Pla2g4a',
'Mapk14',
'Abcd1',
'Brca1',
'Hsd17b4',
'Ndufs6',
'Lipe',
'Fabp2',
'Abcd3',
'Alox12e',
'Cyb5a',
'Ucp3',
'Cyp2c37',
'Cyp2c38',
'Cyp2c39',
'Cyp2c40',
'Sesn2',
'Acot8',
'Plp1',
'Abcc9',
'Pla2g5',
'Lypla1',
'Pex7',
'Ptgs2',
'Acads',
'Acsbg3',
'Fads2b',
'Oxsm',
'Acot4',
'Acsbg2',
'Hadha',
'Echdc2',
'Asah1',
'Acsl5',
'Ptges',
'Hao1',
'Cyp2c29',
'Angptl3',
'Cpt1b',
'Acss1',
'Acsm3',
'Them4',
'Acly',
'Crot',
'Decr1',
'Ptgr1',
'Slc27a5',
'Elovl1',
'C1qtnf9',
'Pla2g4f',
'Pla2g4d',
'Adipor1',
'Cygb',
'Mlycd',
'Prkaa1',
'Fa2h',
'Sco1',
'Acaca',
'Slc27a1',
'Adipoq',
'Abcd2',
'Hadh',
'Gpam',
'Pdpn',
'Cyp1b1',
'Pparg',
'Acot2',
'Prkab2',
'Acot5',
'Scd4',
'Dagla',
'Cyp2c54',
'Eci3',
'Q7TQ05',
'Fads6',
'Gstm7',
'Thnsl2',
'Acsm4',
'Aasdh',
'Ankrd23',
'Acsm5',
'Acnat2',
'Echs1',
'Mblac2',
'Scd3',
'Q8BQ93',
'Adipor2',
'Prkaa2',
'Mtln',
'Q8BW35',
'Acsl4',
'Ptges2',
'Acaa2',
'Pla2g3',
'Pm20d1',
'Q8C168',
'Q8C5J1',
'Acat2',
'Aldh1l2',
'Acsm2',
'Pnpla8',
'Gpat4',
'Hacd3',
'Ltc4s',
'Acad10',
'Acat1',
'Sgpl1',
'Olah',
'Gstm4',
'Hpgd',
'Acaa1b',
'Ces1d',
'Acsf2',
'Pla2g15',
'Acsm1',
'Slc27a4',
'Cbr4',
'Daglb',
'Cyp4a12a',
'Ces1f',
'Baat',
'Cyp2c50',
'Atp6v1b1',
'Pex2',
'Etfdh',
'Acaa1a',
'Myo5a',
'Hadhb',
'Etfa',
'PEDS1',
'Pecr',
'Cyp4f3',
'Nudt7',
'Lpin3',
'Lpin2',
'Acsbg1',
'Mgst3',
'Them5',
'Nudt8',
'Cyp2u1',
'Pex13',
'Aacs',
'Ephx1',
'Hacd2',
'Alkbh7',
'Echdc3',
'Cyp2c55',
'Aig1',
'Prxl2b',
'Abhd5',
'Ehhadh',
'Acoxl',
'Cyp4v2',
'Cyp2s1',
'Mecr',
'Etfb',
'Cyp4f14',
'Elovl4',
'Hpgds',
'Ivd',
'Cyp2d22',
'Hao2',
'Pla2g10',
'Abcb11',
'Qki',
'Adh4',
'Pla2g2f',
'Prkab1',
'Acot9',
'Lypla2',
'Eci2',
'Lipa',
'Pla2g1b',
'Nr1h3',
'Ggt5']

In [56]:
sc.tl.score_genes(cmc,
                  gene_list = fatty_acid_genes,
                  score_name = 'fatty_acid_score',
                  random_state = 1712)

In [57]:
sc.pl.umap(cmc, 
           frameon = False, 
           color = [ 'genotype', 'cell_state','fatty_acid_score'], 
           size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r')

In [58]:
present_fatty_acid_genes = cmc.var.index[cmc.var.index.isin(fatty_acid_genes)]
present_fatty_acid_genes = present_fatty_acid_genes.astype(str)
present_fatty_acid_genes

In [59]:
expression_matrix = (cmc[:, present_fatty_acid_genes].X > 0).toarray()
expression_df = pd.DataFrame(expression_matrix, index=cmc.obs.index, columns=present_fatty_acid_genes)
expression_df['cell_state'] = cmc.obs['cell_state'].values

fraction_expressed = expression_df.groupby('cell_state').mean()


genes_above_threshold = (fraction_expressed.max(axis=0) > 0.4)
filtered_fatty_acid_genes = present_fatty_acid_genes[genes_above_threshold.values]

In [60]:
sc.pl.dotplot(cmc, filtered_fatty_acid_genes,groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma',title = 'fatty_acid_cmcs_cell_state', save= 'fatty_acid_cmcs_cell_state.png')

In [61]:
sc.pl.dotplot(cmc,filtered_fatty_acid_genes,groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='fatty_acid_cmcs_genotype',save= 'fatty_acid_cmcs_genotype.png')

# Genes from GSEA

## Apoptosis

In [62]:
apoptotic_gsea = ['Add1',
'Aifm3',
'Ankh',
'Anxa1',
'App',
'Atf3',
'Avpr1A',
'Bax',
'Bcap31',
'Bcl10',
'Bcl2L1',
'Bcl2L10',
'Bcl2L11',
'Bcl2L2',
'Bgn',
'Bid',
'Bik',
'Birc3',
'Bmf',
'Bmp2',
'Bnip3L',
'Brca1',
'Btg2',
'Btg3',
'Casp1',
'Casp2',
'Casp3',
'Casp4',
'Casp6',
'Casp7',
'Casp8',
'Casp9',
'Cav1',
'Ccna1',
'Ccnd1',
'Ccnd2',
'Cd14',
'Cd2',
'Cd38',
'Cd44',
'Cd69',
'Cdc25B',
'Cdk2',
'Cdkn1A',
'Cdkn1B',
'Cflar',
'Clu',
'Crebbp',
'Cth',
'Ctnnb1',
'Cyld',
'Dap',
'Dap3',
'Dcn',
'Ddit3',
'Dffa',
'Diablo',
'Dnaja1',
'Dnajc3',
'Dnm1L',
'Dpyd',
'Ebp',
'Egr3',
'Emp1',
'Eno2',
'Erbb2',
'Erbb3',
'Ereg',
'Etf1',
'F2',
'F2R',
'Fas',
'Faslg',
'Fdxr',
'Fez1',
'Gadd45A',
'Gadd45B',
'Gch1',
'Gna15',
'Gpx1',
'Gpx3',
'Gpx4',
'Gsn',
'Gsr',
'Gstm1',
'Gucy2D',
'H1-0',
'Hgf',
'Hmgb2',
'Hmox1',
'Hspb1',
'Ier3',
'Ifitm3',
'Ifnb1',
'Ifngr1',
'Igf2R',
'Igfbp6',
'Il18',
'Il1A',
'Il1B',
'Il6',
'Irf1',
'Isg20',
'Jun',
'Krt18',
'Lef1',
'Lgals3',
'Lmna',
'Plppr4',
'Lum',
'Madd',
'Mcl1',
'Mgmt',
'Mmp2',
'Nedd9',
'Nefh',
'Pak1',
'Pdcd4',
'Pdgfrb',
'Pea15',
'Plat',
'Plcb2',
'Pmaip1',
'Ppp2R5B',
'Ppp3R1',
'Ppt1',
'Prf1',
'Psen1',
'Psen2',
'Ptk2',
'Rara',
'Rela',
'Retsat',
'Rhob',
'Rhot2',
'Rnasel',
'Rock1',
'Sat1',
'Satb1',
'Sc5D',
'Slc20A1',
'Smad7',
'Sod1',
'Sod2',
'Sptan1',
'Sqstm1',
'Tap1',
'Tgfb2',
'Tgfbr3',
'Timp1',
'Timp2',
'Timp3',
'Tnf',
'Tnfrsf12A',
'Tnfsf10',
'Top2A',
'Tspo',
'Txnip',
'Vdac2',
'Wee1',
'Xiap']

In [63]:
sc.tl.score_genes(cmc, gene_list= apoptotic_gsea, score_name= 'apoptotic_gsea')

In [64]:
sc.pl.umap(cmc, color= ['sample','genotype','apoptotic_gsea'],size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r', frameon = False)

In [65]:
present_apoptotic_gsea = cmc.var.index[cmc.var.index.isin(apoptotic_gsea)]
present_apoptotic_gsea = present_apoptotic_gsea.astype(str)
present_apoptotic_gsea

In [66]:
sc.pl.dotplot(cmc, present_apoptotic_gsea, groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='apoptosis_gsea_cmcs_genotype',save= 'apoptosis_gsea_cmcs_genotype')

In [67]:
sc.pl.dotplot(cmc, present_apoptotic_gsea, groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma', title='apoptosis_gsea_cmcs_cell_state',save= 'apoptosis_gsea_cmcs_cell_state')

In [68]:
sc.pl.dotplot(cmc, present_apoptotic_gsea, groupby= ['genotype','cell_state'],
              dendrogram = True,
              cmap = 'magma', title='apoptosis_gsea_cmcs_genotype_cell_state',save= 'apoptosis_gsea_cmcs_genotype_cell_state')

## Interferon gaama

In [69]:
interferon_gaama_gsea = ['Adar',
'Apol6',
'Arid5B',
'Arl4A',
'Auts2',
'B2M',
'Bank1',
'Batf2',
'Bpgm',
'Bst2',
'Btg1',
'C1R',
'C1S',
'Casp1',
'Casp3',
'Casp4',
'Casp7',
'Casp8',
'Ccl2',
'Ccl5',
'Ccl7',
'Cd274',
'Cd38',
'Cd40',
'Cd69',
'Cd74',
'Cd86',
'Cdkn1A',
'Cfb',
'Cfh',
'Ciita',
'Cmklr1',
'Cmpk2',
'Csf2Rb',
'Cxcl10',
'Cxcl11',
'Cxcl9',
'Rigi',
'Ddx60',
'Dhx58',
'Eif2Ak2',
'Eif4E3',
'Epsti1',
'Fas',
'Fcgr1A',
'Fgl2',
'Fpr1',
'Cmtr1',
'Gbp4',
'Gbp6',
'Gch1',
'Gpr18',
'Gzma',
'Herc6',
'Hif1A',
'Hla-A',
'Hla-B',
'Hla-Dma',
'Hla-Dqa1',
'Hla-Drb1',
'Hla-G',
'Icam1',
'Ido1',
'Ifi27',
'Ifi30',
'Ifi35',
'Ifi44',
'Ifi44L',
'Ifih1',
'Ifit1',
'Ifit2',
'Ifit3',
'Ifitm2',
'Ifitm3',
'Ifnar2',
'Il10Ra',
'Il15',
'Il15Ra',
'Il18Bp',
'Il2Rb',
'Il4R',
'Il6',
'Il7',
'Irf1',
'Irf2',
'Irf4',
'Irf5',
'Irf7',
'Irf8',
'Irf9',
'Isg15',
'Isg20',
'Isoc1',
'Itgb7',
'Jak2',
'Klrk1',
'Lap3',
'Lats2',
'Lcp2',
'Lgals3Bp',
'Ly6E',
'Lysmd2',
'Marchf1',
'Tmt1B',
'Mt2A',
'Mthfd2',
'Mvp',
'Mx1',
'Mx2',
'Myd88',
'Nampt',
'Ncoa3',
'Nfkb1',
'Nfkbia',
'Nlrc5',
'Nmi',
'Nod1',
'Nup93',
'Oas2',
'Oas3',
'Oasl',
'Ogfr',
'P2Ry14',
'Parp12',
'Parp14',
'Pde4B',
'Peli1',
'Pfkp',
'Pim1',
'Pla2G4A',
'Plscr1',
'Pml',
'Pnp',
'Pnpt1',
'Helz2',
'Psma2',
'Psma3',
'Psmb10',
'Psmb2',
'Psmb8',
'Psmb9',
'Psme1',
'Psme2',
'Ptgs2',
'Ptpn1',
'Ptpn2',
'Ptpn6',
'Rapgef6',
'Rbck1',
'Ripk1',
'Ripk2',
'Rnf213',
'Rnf31',
'Rsad2',
'Rtp4',
'Samd9L',
'Samhd1',
'Sectm1',
'Selp',
'Serping1',
'Slamf7',
'Slc25A28',
'Socs1',
'Socs3',
'Sod2',
'Sp110',
'Sppl2A',
'Sri',
'Sspn',
'St3Gal5',
'St8Sia4',
'Stat1',
'Stat2',
'Stat3',
'Stat4',
'Tap1',
'Tapbp',
'Tdrd7',
'Tnfaip2',
'Tnfaip3',
'Tnfaip6',
'Tnfsf10',
'Tor1B',
'Trafd1',
'Trim14',
'Trim21',
'Trim25',
'Trim26',
'Txnip',
'Ube2L6',
'Upp1',
'Usp18',
'Vamp5',
'Vamp8',
'Vcam1',
'Wars1',
'Xaf1',
'Xcl1',
'Zbp1',
'Znfx1']

In [70]:
sc.tl.score_genes(cmc, gene_list= interferon_gaama_gsea, score_name= 'interferon_gaama_gsea')

In [71]:
sc.pl.umap(cmc, color= ['sample','genotype','interferon_gaama_gsea'],size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r', frameon = False)

In [72]:
present_interferon_gaama_gsea = cmc.var.index[cmc.var.index.isin(interferon_gaama_gsea)]
high_score_cells = cmc.obs[cmc.obs['interferon_gaama_gsea'] > 0.4].index
filtered_cmc = cmc[high_score_cells, present_interferon_gaama_gsea]
present_interferon_gaama_gsea = filtered_cmc.var.index.astype(str)

present_interferon_gaama_gsea = present_interferon_gaama_gsea.astype(str)
present_interferon_gaama_gsea

In [73]:
sc.pl.dotplot(cmc, present_interferon_gaama_gsea, groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='interferon_gaama_gsea_cmcs_genotype',save= 'interferon_gaama_gsea_cmcs_genotype')

In [74]:
sc.pl.dotplot(cmc, present_interferon_gaama_gsea, groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma', title='interferon_gaama_gsea_cmcs_cell_state',save= 'interferon_gaama_gsea_cmcs_cell_state')

In [75]:
sc.pl.dotplot(cmc, present_interferon_gaama_gsea, groupby= ['genotype','cell_state'],
              dendrogram = True,
              cmap = 'magma', title='interferon_gaama_gsea_cmcs_genotype_cell_state',save= 'interferon_gaama_gsea_cmcs_genotype_cell_state')

## Interferon Alpha

In [76]:
interferon_alpha_gsea = ['Adar',
'B2M',
'Batf2',
'Bst2',
'C1S',
'Casp1',
'Casp8',
'Ccrl2',
'Cd47',
'Cd74',
'Cmpk2',
'Cnp',
'Csf1',
'Cxcl10',
'Cxcl11',
'Ddx60',
'Dhx58',
'Eif2Ak2',
'Elf1',
'Epsti1',
'Mvb12A',
'Tent5A',
'Cmtr1',
'Gbp2',
'Gbp4',
'Gmpr',
'Herc6',
'Hla-C',
'Ifi27',
'Ifi30',
'Ifi35',
'Ifi44',
'Ifi44L',
'Ifih1',
'Ifit2',
'Ifit3',
'Ifitm1',
'Ifitm2',
'Ifitm3',
'Il15',
'Il4R',
'Il7',
'Irf1',
'Irf2',
'Irf7',
'Irf9',
'Isg15',
'Isg20',
'Lamp3',
'Lap3',
'Lgals3Bp',
'Lpar6',
'Ly6E',
'Mov10',
'Mx1',
'Ncoa7',
'Nmi',
'Nub1',
'Oas1',
'Oasl',
'Ogfr',
'Parp12',
'Parp14',
'Parp9',
'Plscr1',
'Pnpt1',
'Helz2',
'Procr',
'Psma3',
'Psmb8',
'Psmb9',
'Psme1',
'Psme2',
'Ripk2',
'Rnf31',
'Rsad2',
'Rtp4',
'Samd9',
'Samd9L',
'Sell',
'Slc25A28',
'Sp110',
'Stat2',
'Tap1',
'Tdrd7',
'Tmem140',
'Trafd1',
'Trim14',
'Trim21',
'Trim25',
'Trim26',
'Trim5',
'Txnip',
'Uba7',
'Ube2L6',
'Usp18',
'Wars1']

In [77]:
sc.tl.score_genes(cmc, gene_list= interferon_alpha_gsea, score_name= 'interferon_alpha_gsea')

In [78]:
sc.pl.umap(cmc, color= ['sample','genotype','interferon_alpha_gsea'],size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r', frameon = False)

In [79]:
present_interferon_alpha_gsea = cmc.var.index[cmc.var.index.isin(interferon_alpha_gsea)]
present_interferon_alpha_gsea = present_interferon_alpha_gsea.astype(str)
present_interferon_alpha_gsea

In [80]:
sc.pl.dotplot(cmc, present_interferon_alpha_gsea, groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='interferon_alpha_gsea_cmcs_genotype',save= 'interferon_alpha_gsea_cmcs_genotype')

In [81]:
sc.pl.dotplot(cmc, present_interferon_alpha_gsea, groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma', title='interferon_alpha_gsea_cmcs_cell_state',save= 'interferon_alpha_gsea_cmcs_cell_state')

In [82]:
sc.pl.dotplot(cmc, present_interferon_alpha_gsea, groupby= ['genotype','cell_state'],
              dendrogram = True,
              cmap = 'magma', title='interferon_alpha_gsea_cmcs_genotype_cell_state',save= 'interferon_alpha_gsea_cmcs_genotype_cell_state')

# KEGG genes

## GOBP_T_CELL_MEDIATED_CYTOTOXICITY

In [83]:
t_cell_cytotoxicity = ['H2-Ea',
'H2-T27',
'Slc22a13',
'Nckap1l',
'H2-Q6',
'H2-Q7',
'H2-M10.3',
'Ager',
'Rab27a',
'B2m',
'Cd1d1',
'Cd1d2',
'Ctsc',
'Ctsh',
'Emp2',
'Muc4',
'Fadd',
'Gzmb',
'H2-Bl',
'H2-D1',
'H2-K1',
'H2-M10.1',
'H2-M2',
'H2-M3',
'H2-M9',
'H2-Q1',
'H2-Q10',
'H2-Q2',
'H2-Q4',
'H2-Q7',
'H2-T10',
'H2-T22',
'H2-T23',
'H2-T24',
'H2-T3',
'Mr1',
'Hfe',
'Hprt1',
'Il12a',
'Il12b',
'Il7r',
'Klrd1',
'Gzmm',
'P2rx7',
'Prf1',
'Pnp',
'Ppp3cb',
'Ptprc',
'Nectin2',
'Raet1d',
'Raet1d',
'Raet1d',
'Serpinb9b',
'Serpinb9',
'Tap2',
'Gfus',
'Cyrib',
'H2-M10.4',
'H2-M11',
'H2-M1',
'H2-M10.5',
'H2-M5',
'Mill2',
'Mill1',
'H2-M10.2',
'Raet1e',
'H2-M10.6',
'Pvr',
'Stx7',
'Ebag9',
'Ripk3',
'Raet1d',
'H60b',
'H2-T26',
'H60c',
'Stx11',
'2410137M14Rik',
'Ulbp1',
'Il23a',]

In [84]:
sc.tl.score_genes(cmc, gene_list= t_cell_cytotoxicity, score_name= 't_cell_cytotoxicity')

In [85]:
sc.pl.umap(cmc, color= ['sample','genotype','t_cell_cytotoxicity'],size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r', frameon = False)

In [86]:
present_t_cell_cytotoxicity = cmc.var.index[cmc.var.index.isin(t_cell_cytotoxicity)]
present_t_cell_cytotoxicity = present_t_cell_cytotoxicity.astype(str)
present_t_cell_cytotoxicity

In [87]:
sc.pl.dotplot(cmc, present_t_cell_cytotoxicity, groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='t_cell_cytotoxicity_cmcs_genotype',save= 't_cell_cytotoxicity_cmcs_genotype')

In [88]:
sc.pl.dotplot(cmc, present_t_cell_cytotoxicity, groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma', title='t_cell_cytotoxicity_cmcs_cell_state',save= 't_cell_cytotoxicity_cmcs_cell_state')

In [89]:
sc.pl.dotplot(cmc, present_t_cell_cytotoxicity, groupby= ['genotype','cell_state'],
              dendrogram = True,
              cmap = 'magma', title='t_cell_cytotoxicity_cmcs_genotype_cell_state',save= 't_cell_cytotoxicity_cmcs_genotype_cell_state')

## ROS Pathway

In [90]:
ros_genes = ['Abcc1',
'Atox1',
'Cat',
'Cdkn2D',
'Egln2',
'Ercc2',
'Fes',
'Ftl',
'G6Pd',
'Gclc',
'Gclm',
'Glrx',
'Glrx2',
'Gpx3',
'Gpx4',
'Gsr',
'Lamtor5',
'Hhex',
'Hmox2',
'Ipcef1',
'Junb',
'Lsp1',
'Mbp',
'Mgst1',
'Mpo',
'Msra',
'Ndufa6',
'Ndufb4',
'Ndufs2',
'Nqo1',
'Oxsr1',
'Pdlim1',
'Pfkp',
'Ptpa',
'Prdx1',
'Prdx2',
'Prdx4',
'Prdx6',
'Prnp',
'Sbno2',
'Scaf4',
'Selenos',
'Sod1',
'Sod2',
'Srxn1',
'Stk25',
'Txn',
'Txnrd1',
'Txnrd2',]

In [91]:
sc.tl.score_genes(cmc, gene_list= ros_genes, score_name= 'ros_score')

In [92]:
sc.pl.umap(cmc, color= ['sample','genotype','ros_score'],size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r', frameon = False)

In [93]:
present_ros_score = cmc.var.index[cmc.var.index.isin(ros_genes)]
present_ros_score = present_ros_score.astype(str)
present_ros_score

In [94]:
sc.pl.dotplot(cmc, present_ros_score, groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='ros_score_cmcs_genotype',save= 'ros_score_cmcs_genotype')

In [95]:
sc.pl.dotplot(cmc, present_ros_score, groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma', title='ros_score_cmcs_cell_state',save= 'ros_score_cmcs_cell_state')

In [96]:
sc.pl.dotplot(cmc, present_ros_score, groupby= ['genotype','cell_state'],
              dendrogram = True,
              cmap = 'magma', title='ros_score_cmcs_genotype_cell_state',save= 'ros_score_cmcs_genotype_cell_state')

## STING mediated

In [97]:
sting_genes = ['Mre11',
'Xrcc5',
'Dtx4',
'Irf3',
'Trim21',
'Nlrp4',
'Ifi16',
'Cgas',
'Stat6',
'Nlrc3',
'Ddx41',
'Tbk1',
'Sting1',
'Xrcc6',
'Trex1',
'Prkdc',
'Sting1',]

In [98]:
sc.tl.score_genes(cmc, gene_list= sting_genes, score_name= 'sting_score')

In [99]:
sc.pl.umap(cmc, color= ['sample','genotype','sting_score'],size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r', frameon = False)

In [100]:
present_sting_score = cmc.var.index[cmc.var.index.isin(sting_genes)]
present_sting_score = present_sting_score.astype(str)
present_sting_score

In [101]:
sc.pl.dotplot(cmc, present_sting_score, groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='sting_score_cmcs_genotype',save= 'sting_score_cmcs_genotype')

In [102]:
sc.pl.dotplot(cmc, present_sting_score, groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma', title='sting_score_cmcs_cell_state',save= 'sting_score_cmcs_cell_state')

In [103]:
sc.pl.dotplot(cmc, present_sting_score, groupby= ['genotype','cell_state'],
              dendrogram = True,
              cmap = 'magma', title='sting_score_cmcs_genotype_cell_state',save= 'sting_score_cmcs_genotype_cell_state')

## Cytosolic DNA Sensing Pathway

In [104]:
cytosolic_dna = ['Adar',
'Polr3F',
'Polr3G',
'Polr3C',
'Ripk3',
'Polr3A',
'Trex1',
'Chuk',
'Polr3H',
'Rigi',
'Pycard',
'Tbk1',
'Sting1',
'Ifna1',
'Ifna2',
'Ifna4',
'Ifna5',
'Ifna6',
'Ifna7',
'Ifna8',
'Ifna10',
'Ifna13',
'Ifna14',
'Ifna16',
'Ifna17',
'Ifna21',
'Ifnb1',
'Ikbkb',
'Il1B',
'Il6',
'Il18',
'Cxcl10',
'Irf3',
'Irf7',
'Ccl4L2',
'Nfkb1',
'Nfkbia',
'Nfkbib',
'Polr1D',
'Polr3K',
'Polr3B',
'Mavs',
'Rela',
'Ccl4',
'Ccl5',
'Polr3D',
'Zbp1',
'Casp1',
'Polr3Gl',
'Ikbkg',
'Ripk1',
'Il33',
'Aim2',
'Polr1C',
'Ccl4L2',
'Ikbke',]

In [105]:
sc.tl.score_genes(cmc, gene_list= cytosolic_dna, score_name= 'cytosolic_dna_score')

In [106]:
sc.pl.umap(cmc, color= ['sample','genotype','cytosolic_dna_score'],size = 2.5, 
           legend_fontsize = 5, 
           ncols = 3,
           cmap = 'RdYlBu_r', frameon = False)

In [107]:
present_cytosolic_dna = cmc.var.index[cmc.var.index.isin(cytosolic_dna)]
present_cytosolic_dna = present_cytosolic_dna.astype(str)
present_cytosolic_dna

In [108]:
sc.pl.dotplot(cmc, present_cytosolic_dna, groupby= 'genotype',
              dendrogram = True,
              cmap = 'magma', title='cytosolic_dna_score_cmcs_genotype',save= 'cytosolic_dna_score_cmcs_genotype')

In [109]:
sc.pl.dotplot(cmc, present_cytosolic_dna, groupby= 'cell_state',
              dendrogram = True,
              cmap = 'magma', title='cytosolic_dna_score_cmcs_cell_state',save= 'cytosolic_dna_score_cmcs_cell_state')

In [110]:
sc.pl.dotplot(cmc, present_cytosolic_dna, groupby= ['genotype','cell_state'],
              dendrogram = True,
              cmap = 'magma', title='cytosolic_dna_score_cmcs_genotype_cell_state',save= 'cytosolic_dna_score_cmcs_genotype_cell_state')

In [111]:
cmc.write_h5ad(f'../data/dmd_cmc_gene_set_scoring_added_{timestamp}.h5ad')

# Overall Visualization

## Genotype - Cell state

In [112]:
sc.pl.umap(cmc, 
           frameon = False, 
           color = ['genotype', 'cell_state','interferon_score','interferon_gaama_gsea','interferon_alpha_gsea','apoptotic_score','apoptotic_gsea','ferroptosis_score','phosphorylation_score','glycolysis_score','pyroptosis_score','fatty_acid_score','sting_score','ros_score','t_cell_cytotoxicity','cytosolic_dna_score'], 
           size = 1, 
           legend_fontsize = 5, 
           ncols = 5,
           cmap = 'RdYlBu_r')

In [113]:
df = cmc.obs[['genotype', 'cell_state', 'interferon_score','interferon_alpha_gsea','interferon_gaama_gsea', 'apoptotic_score','apoptotic_gsea', 'ferroptosis_score', 'phosphorylation_score','glycolysis_score','pyroptosis_score','fatty_acid_score','sting_score','ros_score','t_cell_cytotoxicity','cytosolic_dna_score']]
mean = df.groupby(['cell_state', 'genotype']).mean()
mean

In [114]:
def compute_means(group):
    result = {}
    for column in group.columns:
        if column not in ['cell_state', 'genotype']:
            positive_values = group[column][group[column] > 0]
            negative_values = group[column][group[column] < 0]
            positive_mean = positive_values.mean() if not positive_values.empty else 0
            negative_mean = negative_values.mean() if not negative_values.empty else 0
            result[f'{column}_Positive_Mean'] = positive_mean
            result[f'{column}_Negative_Mean'] = negative_mean
    return pd.Series(result)

means = df.groupby(['cell_state', 'genotype']).apply(compute_means).reset_index()
means

In [115]:
means.to_csv('../gene_set_scoring/mean_gene_set_scores_cellstate_genotype.csv')

In [116]:
means = pd.DataFrame(means)

positive_means = means.filter(like='_Positive_Mean')
negative_means = means.filter(like='_Negative_Mean')

positive_means = positive_means.set_index(means[['cell_state', 'genotype']].apply(lambda x: '_'.join(x), axis=1))
negative_means = negative_means.set_index(means[['cell_state', 'genotype']].apply(lambda x: '_'.join(x), axis=1))

positive_means = positive_means.apply(pd.to_numeric, errors='coerce')
negative_means = negative_means.apply(pd.to_numeric, errors='coerce')

positive_means = positive_means.fillna(0)
negative_means = negative_means.fillna(0)

In [117]:
sns.set(style='white') 
g = sns.clustermap(positive_means, 
                   cmap='RdYlBu_r', 
                   annot=True)  

for ax in g.ax_row_dendrogram, g.ax_col_dendrogram:
    ax.grid(False)

plt.title('Positive Means of Scoring', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [118]:
sns.set(style='white') 
g = sns.clustermap(negative_means, 
                   cmap='RdYlBu_r', 
                   annot=True)  

for ax in g.ax_row_dendrogram, g.ax_col_dendrogram:
    ax.grid(False)

plt.title('Negative Means of Scoring', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

## Only genotype

In [119]:
df = cmc.obs[['genotype', 'interferon_score', 'interferon_alpha_gsea', 'interferon_gaama_gsea',
              'apoptotic_score', 'apoptotic_gsea', 'ferroptosis_score', 'phosphorylation_score',
              'glycolysis_score', 'pyroptosis_score', 'fatty_acid_score', 'sting_score',
              'ros_score', 't_cell_cytotoxicity', 'cytosolic_dna_score']]
mean = df.groupby('genotype').mean()
mean

In [120]:
def compute_means(group):
    result = {}
    for column in group.columns:
        if column not in [ 'genotype']:
            positive_values = group[column][group[column] > 0]
            negative_values = group[column][group[column] < 0]
            positive_mean = positive_values.mean() if not positive_values.empty else 0
            negative_mean = negative_values.mean() if not negative_values.empty else 0
            result[f'{column}_Positive_Mean'] = positive_mean
            result[f'{column}_Negative_Mean'] = negative_mean
    return pd.Series(result)

means = df.groupby(['genotype']).apply(compute_means).reset_index()
means

In [121]:
means.to_csv('../gene_set_scoring/mean_gene_set_scores_genotype_only.csv')

In [122]:
means = pd.DataFrame(means)

positive_means = means.filter(like='_Positive_Mean')
negative_means = means.filter(like='_Negative_Mean')

positive_means = positive_means.set_index(means[['genotype']].apply(lambda x: '_'.join(x), axis=1))
negative_means = negative_means.set_index(means[['genotype']].apply(lambda x: '_'.join(x), axis=1))

positive_means = positive_means.apply(pd.to_numeric, errors='coerce')
negative_means = negative_means.apply(pd.to_numeric, errors='coerce')

positive_means = positive_means.fillna(0)
negative_means = negative_means.fillna(0)

In [123]:
sns.set(style='white') 
g = sns.clustermap(positive_means, 
                   cmap='RdYlBu_r', 
                   annot=True)  

for ax in g.ax_row_dendrogram, g.ax_col_dendrogram:
    ax.grid(False)

plt.title('Positive Means of Scoring', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [124]:
sns.set(style='white') 
g = sns.clustermap(negative_means, 
                   cmap='RdYlBu_r', 
                   annot=True)  

for ax in g.ax_row_dendrogram, g.ax_col_dendrogram:
    ax.grid(False)

plt.title('Negative Means of Scoring', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()